In [ ]:
import sqlfile
import sqlalchemy
import pandas as pd
import csv
from collections import Counter
import model


def change_data_format(res):
    temp = res.fetchall()

    with open('file_for_edit.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(res.keys())
        for row in temp:
            writer.writerow(row)

    name_new_file = model.work_with_info('file_for_edit.csv', 'file_for_edit_new.csv')
    return name_new_file


def conclusion(pred):
    total = 0
    for item in pred:
        total += item

    return 'female' if total < 0 else 'male'


def work_with_model(session_id, file_name):
    new_df = pd.read_csv(file_name)
    new_model = model.model_preparation()
    predict = model.predict_for_one_session(new_model, session_id, new_df)
    total = conclusion(predict)
    return total


def info_about_users(session_id, file_name, version=None, info_in_percent=False):
    number = 0
    all_info = list()
    new_list_with_percent = list()

    new_df = pd.read_csv(file_name)

    # получаем количество записей по session_id
    count_of_records = new_df.iloc[-1, 1]

    list_with_category = ['category_a', 'category_b', 'category_c', 'category_d']
    for item in list_with_category:
        inf = new_df[item][:]
        count = dict(Counter(inf.values))
        all_info.append(count)

    if version == 'percent':
        print(f'Категории пользователя с session_id = {session_id}')
        for mas in all_info:
            temp = dict()
            print('Категория: ', list_with_category[number])
            for key, value in mas.items():
                print(key, ': ', value*100/count_of_records, '%')
                temp[key] = (value*100/count_of_records)/100

            new_list_with_percent.append(temp)
            number += 1
            print('----------------------')
        return new_list_with_percent

    elif version == 'avg':
        print(f'Категории пользователя с session_id = {session_id}')
        for mas in all_info:
            print('Категория: ', list_with_category[number])
            for key, value in mas.items():
                print(key, ': ', value)
            number += 1
            print('----------------------')

        return all_info
    pass


def sql_request(session_id, version):
    result = sqlfile.get_info_about_one_session(session_id)
    file_name = change_data_format(result)

    info_about_users(session_id, 'file_for_edit.csv', version=version, info_in_percent=True)
    total = work_with_model(session_id, file_name)

    return total


def get_data(session_id, version=None):
    res = sqlfile.get_info_about_one_session(session_id)
    a = change_data_format(res)
    file_name = 'file_for_edit.csv'
    data = info_about_users(session_id, file_name, version=version)
    return data


def main():
    pass


if __name__ == '__main__':
    main()